# 12 激活函数与损失函数

## 激活函数
<img src='./images/09.png' style='zoom:50%'>

### sigmoid / logistic

$$f(x)=\delta(x)=\frac{1}{1+e^{-x}}$$  
<img src='./images/10.png' style='zoom:75%'>
导数形式：$\delta'(x)=\delta(1-\delta)$ ，这个函数在较大值时，导数基本为0，梯度下降不会得到很好的更新。

In [1]:
import torch
import torch.nn.functional as F           #提取sigmoid等函数

In [2]:
a = torch.linspace(-100,100,10)
a

tensor([-100.0000,  -77.7778,  -55.5556,  -33.3333,  -11.1111,   11.1111,
          33.3333,   55.5555,   77.7778,  100.0000])

In [3]:
torch.sigmoid(a)

tensor([0.0000e+00, 1.6655e-34, 7.4564e-25, 3.3382e-15, 1.4945e-05, 9.9999e-01,
        1.0000e+00, 1.0000e+00, 1.0000e+00, 1.0000e+00])

### tanh (在RNN中应用较多）

$$f(x)=tanh(x)=\frac{e^x-e^{-x}}{e^x+e^{-x}}=2sigmoid(2x)-1$$

$$f'(x)=1-tanh^2(x)$$  
<img src='./images/11.png' style='zoom:75%'>

In [4]:
a = torch.linspace(-1,1,10)
torch.tanh(a)

tensor([-0.7616, -0.6514, -0.5047, -0.3215, -0.1107,  0.1107,  0.3215,  0.5047,
         0.6514,  0.7616])

### Rectified Linear Unit =Relu (常用)

$$f(x)=\begin{cases} 
		0, & for\ x<0\\ 
		x, & for\ x\ge0 
	\end{cases}$$  
<img src='./images/12.png' style='zoom:75%'>

In [5]:
a = torch.linspace(-1,1,10)
torch.relu(a)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111, 0.3333, 0.5556, 0.7778,
        1.0000])

In [6]:
F.relu(a)

tensor([0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.1111, 0.3333, 0.5556, 0.7778,
        1.0000])

## 损失函数

### MSE
$loss = \sum [y-(wx+b)]^2$  
$loss = (norm(y-(wx+b)))^2$

### autograd.grad

In [7]:
import torch
import torch.nn.functional as F

In [8]:
x = torch.ones(1)
w = torch.full([1],2)
mse = F.mse_loss(x*w, torch.ones(1))
mse

tensor(1.)

In [9]:
#经过如下操作之后才可以对w自动求导
w.requires_grad_()    #说明w是需要更新的,或者可以使用w=torch.tensor([1.],requires_grad=True)
mse = F.mse_loss(x*w, torch.ones(1))     #重新更新计算图

In [10]:
#此时就可以对w进行自动求导了
torch.autograd.grad(mse,[w])

(tensor([2.]),)

### loss.backward

In [11]:
mse = F.mse_loss(x*w, torch.ones(1))
mse.backward()      #反向传播，计算所有的梯度，需要哪个调用哪个就行

In [12]:
w.grad

tensor([2.])

### Gradient API
###### 两种方式：
* 1.torch.autograd.grad(loss,[w1,w2,...]  
* 2.loss.backward()

### softmax——activate function
$$S(y_i)=\frac{e^{y_i}}{\sum_je^{y_i}}$$  
$$\dfrac{\partial p_i}{\partial a_j}=\begin{cases}p_j(1-p_j)\  ,&i=j\\-p_j\cdot{p_i}\  ,&i\ne j\end{cases}$$

In [13]:
a = torch.rand(3)
a.requires_grad_()

tensor([0.1757, 0.1738, 0.9258], requires_grad=True)

In [14]:
p=F.softmax(a, dim=0)

In [15]:
p.backward                  #backward不能连续使用两次

<bound method Tensor.backward of tensor([0.2430, 0.2425, 0.5145], grad_fn=<SoftmaxBackward>)>

In [16]:
torch.autograd.grad(p[1],[a],retain_graph=True)  #损失函数只能是一个数值，所以这里使用的是p[1],retain_graph=True,
                                                #保证计算图不消除，可以继续调用

(tensor([-0.0589,  0.1837, -0.1248]),)

In [17]:
torch.autograd.grad(p[2],[a])          #可以看出来i=j的时候导数是正的，其他是负的。

(tensor([-0.1250, -0.1248,  0.2498]),)